In [1]:
import numpy as np

In [2]:
# fixed functions
mapping_table = {
    # codigo de grey
    (0,0,0,0) : -3-3j,
    (0,0,0,1) : -3-1j,
    (0,0,1,0) : -3+3j,
    (0,0,1,1) : -3+1j,
    (0,1,0,0) : -1-3j,
    (0,1,0,1) : -1-1j,
    (0,1,1,0) : -1+3j,
    (0,1,1,1) : -1+1j,
    (1,0,0,0) :  3-3j,
    (1,0,0,1) :  3-1j,
    (1,0,1,0) :  3+3j,
    (1,0,1,1) :  3+1j,
    (1,1,0,0) :  1-3j,
    (1,1,0,1) :  1-1j,
    (1,1,1,0) :  1+3j,
    (1,1,1,1) :  1+1j
    }

def QAM_mod(bits):
    return np.array([mapping_table[tuple(b)] for b in bits])

def QAM_demod(symbols):
    demapping_table = {v : k for k, v in mapping_table.items()}
    # array of possible constellation points
    constellation = np.array([x for x in demapping_table.keys()])

    # calculate distance of each RX point to each possible point
    dists = abs(symbols.reshape((-1,1)) - constellation.reshape((1,-1)))

    # for each element in QAM, choose the index in constellation
    # that belongs to the nearest constellation point
    const_index = dists.argmin(axis=1)

    # get back the real constellation point
    hardDecision = constellation[const_index]

    # transform the constellation point into the bit groups
    return np.vstack([demapping_table[C] for C in hardDecision])#, hardDecision

def IDFT(OFDM_data):
    return np.fft.ifft(OFDM_data)

def addCP(OFDM_time):
    cp = OFDM_time[-CP:]               # take the last CP samples ...
    return np.hstack([cp, OFDM_time])  # ... and add them to the beginning

def removeCP(signal):
    return signal[CP:]

# Función para simular la transmisión por el canal
def simulate_channel_transmission(data_sequence, channel_response, SNR_dB=-1):
    # Aplicar la respuesta al impulso del canal mediante la convolución
    transmitted_signal = np.convolve(data_sequence, channel_response, mode='same')

    if SNR_dB == -1:
      return transmitted_signal

    # Calcular la potencia de la señal transmitida
    signal_power = np.mean(np.abs(transmitted_signal) ** 2)

    # Calcular la potencia del ruido a partir de la relación señal a ruido (SNR)
    noise_power = signal_power / (10 ** (SNR_dB / 10))

    # Generar ruido gaussiano con la potencia calculada
    noise = np.random.normal(0, np.sqrt(noise_power), len(transmitted_signal))

    # Agregar ruido a la señal transmitida
    received_signal = transmitted_signal + noise

    return received_signal

def response_rayleigh_channel(symbol_matrix, fading_param = 0.5 + 0.5j):
    # Generar muestras de canal Rayleigh
    channel_gain = np.sqrt(0.5) * (np.random.randn(*symbol_matrix.shape) + 1j * np.random.randn(*symbol_matrix.shape))

    # Aplicar desvanecimiento temporal a cada subportadora
    faded_symbols = symbol_matrix * channel_gain * fading_param

    return faded_symbols

def add_noise(signal, SNR_db):
    # Calcular la potencia de la señal transmitida
    signal_power = np.mean(np.abs(faded_symbols) ** 2)

    # Calcular la potencia del ruido a partir de la relación señal a ruido (SNR)
    noise_power = signal_power / (10 ** (SNR_dB / 10))

    # Generar ruido gaussiano con la potencia calculada
    noise = np.random.normal(0, np.sqrt(noise_power), len(signal))

    # Agregar ruido a la señal transmitida
    noise_signal = signal + noise

    return noise_signal

def simulate_rayleigh_channel_transmission(data_sequence, SNR_db = -1):
  received_signal = response_rayleigh_channel(data_sequence)
  if SNR_db == -1:
    return received_signal
  return add_noise(received_signal, SNR_db)


In [3]:
# in progress functions
def channel_estimate_block(received_signal, pilot_value_mod, method = None):
  if method == 'LS': ## revisar, no dan las dimensiones. devuelve los factores para una linea recta no para cada portadora
    return np.linalg.lstsq(np.vstack((pilot_value_mod.real, pilot_value_mod.imag)).T, received_signal, rcond=None)[0]
  if method == 'MMSE': ## revisar, no secomo elegir la varianza de ruido y los resultados dan muy errados
    noise_cov = 0.000001 * np.identity(len(received_signal))
    channel_cov = np.outer(pilot_value_mod.conj(), pilot_value_mod)
    mmse_gain = np.linalg.pinv(channel_cov + noise_cov) @ pilot_value_mod.conj()
    return received_signal * mmse_gain
  if method == None:
    return received_signal / pilot_value_mod

In [4]:
N = 128 # numero de subportadoras
P = 8 # cantidad de pilotos por bloque
S = 1000 # cantidad de simbolos totales a transmitir (incluyendo pilotos)
bps = 4 # bits por simbolo | modulacion 16-QAM por portadora
pilot_value = 1 # valor de los pilotos
CP = N // 4 # prefijo ciclico

# Para siempre generar los mimsos numeros aleatorios y tener repetibilidad
np.random.seed(123)

In [5]:
# bloque

# genero los S simbolos, combinados con pilotos, donde cada P simbolos con datos se envia un simbolo con solo informacion de pilotos

all_symbols = list(range(S))
pilot_symbols = list(range(0, S, P))
data_symbols = np.delete(all_symbols, pilot_symbols)
all_symbols = np.empty(len(all_symbols), dtype=object)
pilot_symbol = np.ones(N*bps, dtype = int) * pilot_value
for i in pilot_symbols:
  all_symbols[i] = pilot_symbol
for i in data_symbols:
  all_symbols[i] = np.random.randint(2, size=N*bps)

# transmito los S simbolos por el canal de rayleigh

transmitted_signals = np.array([addCP(np.fft.ifft(QAM_mod(symbol.reshape(-1, bps)))) for symbol in all_symbols])
#received_signals = simulate_rayleigh_channel_transmission(transmitted_signals)
received_signals = [simulate_channel_transmission(transmitted_signal, [1], -1) for transmitted_signal in transmitted_signals]

# recupero simbolos de la señal recibida
received_symbols = [np.fft.fft(removeCP(signal)) for signal in received_signals]


# proceso señal recibida, estimo el canal y extraigo los bits enviados
est_symbols = []
pilot_value_mod = QAM_mod(all_symbols[0].reshape(-1,bps))

for i in range(len(received_symbols)):
  if i in pilot_symbols:
    ## estimate channel
    #H_est = channel_estimate_block(received_symbols, pilot_value_mod)
    H_est = np.fft.fft([1], N)
    equalized_symbol = received_symbols[i] / H_est
    est_symbols.append(QAM_demod(equalized_symbol).reshape(-1,))
  if i in data_symbols:
    equalized_symbol = received_symbols[i] / H_est
    est_symbols.append(QAM_demod(equalized_symbol).reshape(-1,))

est_symbols = np.array(est_symbols)

# calculo error entre los bits enviados y recibidos
errors = 0
for i in range(S):
  if i not in pilot_symbols:
    bits = all_symbols[i]
    est_bits = est_symbols[i]
    errors += np.sum(abs(bits-est_bits))
BER = errors / (len(data_symbols) * N * bps)


In [6]:
### tests channel estimate

In [7]:
H_est_none = channel_estimate_block(received_symbols[0], pilot_value_mod)
H_est_ls = channel_estimate_block(received_symbols[0], pilot_value_mod, 'LS')
H_est_mmse = channel_estimate_block(received_symbols[0], pilot_value_mod, 'MMSE')

In [8]:
H_est = np.fft.fft([1], N)

In [9]:
H_est_ls

array([0.5+0.5j, 0.5+0.5j])

In [10]:
### aux para cuando toque hacer peine

parallel_bits = bits.reshape(-1, bps) # paralelizo datos para mandarlos por subportadoras
mod = QAM_mod(parallel_bits) # modulo cada subportadora con 16 QAM

# channel_response = np.random.normal(0, 1, 10)   # Respuesta al impulso del canal (ejemplo aleatorio)

# bloque

# combino los datos a transmitir con los pilotos, completando los simbolos totales a transmitir
all_carriers = list(range(S*N))
pilot_carriers = list(range(0, S*N, P))
data_carriers = np.delete(all_carriers, pilot_carriers)
transmitted_symbols = np.empty(len(all_carriers), dtype=complex)

transmitted_symbols[pilot_carriers] = pilot_value

mod_index = 0
for i in range(S):
    if i not in pilot_carriers:
        transmitted_symbols[i] = mod[mod_index]
        mod_index += 1

IndexError: index 128 is out of bounds for axis 0 with size 128